### ML.NET Binary Classification
Creates a binary classification model to predict the quality of wine using 11 physicochemical features

### NuGet package installation

In [1]:
#r "nuget:Microsoft.ML, 1.4.0"
#r "nuget:XPlot.Plotly, 3.0.1"

Installing package Microsoft.ML, version 1.4.0..............................done!

Successfully added reference to package Microsoft.ML, version 1.4.0

Installing package XPlot.Plotly, version 3.0.1.......done!

Successfully added reference to package XPlot.Plotly, version 3.0.1

### Namespaces

In [2]:
using Microsoft.ML;
using Microsoft.ML.Data;
using Microsoft.ML.Trainers;
using Microsoft.ML.Transforms;
using XPlot.Plotly;

### Input class definition

In [3]:
public class BinaryClassificationData
{
    [LoadColumn(0)]
    public float FixedAcidity;

    [LoadColumn(1)]
    public float VolatileAcidity;

    [LoadColumn(2)]
    public float CitricAcid;

    [LoadColumn(3)]
    public float ResidualSugar;

    [LoadColumn(4)]
    public float Chlorides;

    [LoadColumn(5)]
    public float FreeSulfurDioxide;

    [LoadColumn(6)]
    public float TotalSulfurDioxide;

    [LoadColumn(7)]
    public float Density;

    [LoadColumn(8)]
    public float Ph;

    [LoadColumn(9)]
    public float Sulphates;

    [LoadColumn(10)]
    public float Alcohol;

    [LoadColumn(11)]
    public float Quality;
}

public class RichBinaryClassificationData: BinaryClassificationData
{
    public bool Label => Quality > 5;
}

### Output class definition

In [4]:
public class BinaryClassificationPrediction
{
    public bool Label;

    [ColumnName("PredictedLabel")]
    public bool PredictedLabel;

    public int LabelAsNumber => PredictedLabel ? 1 : 0;
}

### Read the raw data

In [5]:
var mlContext = new MLContext(seed: null);

var trainingData = mlContext.Data.LoadFromTextFile<BinaryClassificationData>(
    "./WineQuality_White_Train.csv", 
    separatorChar: ';',
    hasHeader: true);

### Prepare the data

In [6]:
// Read the raw data into an IDataView, 
// then export to an IEnumerable of the richer C# class to calculate the Label,
// then convert back back into an IDataView.

var stronglyTypedTrainingData = mlContext.Data.CreateEnumerable<RichBinaryClassificationData>(trainingData, false);
trainingData = mlContext.Data.LoadFromEnumerable(stronglyTypedTrainingData);

// Define the pipeline.
var pipeline =
        mlContext.Transforms.ReplaceMissingValues(
            outputColumnName: "FixedAcidity",
            replacementMode: MissingValueReplacingEstimator.ReplacementMode.Mean)
        .Append(mlContext.Transforms.Concatenate("Features",
            new[]
            {
                "FixedAcidity",
                "VolatileAcidity",
                "CitricAcid",
                "ResidualSugar",
                "Chlorides",
                "FreeSulfurDioxide",
                "TotalSulfurDioxide",
                "Density",
                "Ph",
                "Sulphates",
                "Alcohol"
            }))
        .Append(mlContext.BinaryClassification.Trainers.LbfgsLogisticRegression());

### Train the model

In [7]:
var model = pipeline.Fit(trainingData);

### Evaluate the model

In [8]:
// Load the raw test data.
var testData = mlContext.Data.LoadFromTextFile<BinaryClassificationData>(
    "./WineQuality_White_Test.csv", 
    separatorChar: ';',
    hasHeader: true);
    
// Calculate the Label using the same manipulation: IDataView to IEnumerable to IDataView.    
var stronglyTypedTestData = mlContext.Data.CreateEnumerable<RichBinaryClassificationData>(trainingData, false);
testData = mlContext.Data.LoadFromEnumerable(stronglyTypedTestData);

// Score the test data and calculate the metrics.
var scoredData = model.Transform(testData);
var qualityMetrics =  mlContext.BinaryClassification.Evaluate(scoredData);
display(qualityMetrics);

LogLoss,LogLossReduction,Entropy,AreaUnderRocCurve,Accuracy,PositivePrecision,PositiveRecall,NegativePrecision,NegativeRecall,F1Score,AreaUnderPrecisionRecallCurve,ConfusionMatrix
0.7452045259897784,0.1974004033343476,0.928488537853348,0.7909630566845476,0.7390860352310442,0.764524948735475,0.8704280155642024,0.6639757820383451,0.4884929472902747,0.8140465793304221,0.8749940309174482,"{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.764524948735475, 0.6639757820383451 ], PerClassRecall: [ 0.8704280155642024, 0.4884929472902747 ], Counts: [ [ 2237, 333 ], [ 689, 658 ] ], NumberOfClasses: 2 }"


### Visualize the quality metrics

In [9]:
string[] metricNames = 
    { 
        "Log Loss", 
        "Log Loss Reduction", 
        "Entropy", 
        "Area Under Curve", 
        "Accuracy",
        "Positive Recall", 
        "Negative Recall",
        "F1 Score"
    };

double[] metricValues = 
    { 
        qualityMetrics.LogLoss, 
        qualityMetrics.LogLossReduction, 
        qualityMetrics.Entropy, 
        qualityMetrics.AreaUnderRocCurve, 
        qualityMetrics.Accuracy,
        qualityMetrics.PositiveRecall, 
        qualityMetrics.NegativeRecall,
        qualityMetrics.F1Score
    };

var graph = new Graph.Bar()
{
    x = metricValues,
    y = metricNames,
    orientation = "h",
    marker = new Graph.Marker { color = "darkred" }
};

var chart = Chart.Plot(graph);

var layout = new Layout.Layout(){ title="Quality Metrics" };
chart.WithLayout(layout);

display(chart);

### Create a prediction engine and use it on a random sample

In [10]:
// Create prediction engine
var predictionEngine = mlContext.Model.CreatePredictionEngine<RichBinaryClassificationData, BinaryClassificationPrediction>(model);

// Get a random data sample
var shuffledData = mlContext.Data.ShuffleRows(trainingData);
var rawSample = mlContext.Data.TakeRows(shuffledData, 1);
var sample = mlContext.Data.CreateEnumerable<RichBinaryClassificationData>(rawSample, false).First();
display(sample);

// Predict quality of sample
var prediction = predictionEngine.Predict(sample);
display(prediction);

Label,FixedAcidity,VolatileAcidity,CitricAcid,ResidualSugar,Chlorides,FreeSulfurDioxide,TotalSulfurDioxide,Density,Ph,Sulphates,Alcohol,Quality
True,7.8,0.25,0.41,3.7,0.042,37,149,0.9954,3.36,0.45,10,6


LabelAsNumber,Label,PredictedLabel
1,True,True
